In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
!ls /content/drive

MyDrive


In [77]:
!ls /content/drive/MyDrive

 02120546-2022-AP-RMFE.gdoc
'02120546-2022-AP RMFE（副本）.gdoc'
 1.18-1.21账单.gsheet
'1.3-1.10 账单.gsheet'
 2021-12-03_inventory.pdf.zip
 2023计划表.gsheet
'2.21 small pre.gdoc'
'2 8 meeting notes.gdoc'
'Ali Template Preliminary Round.gslides'
'Application Record.gsheet'
'Asset Magt-Yifei Li.gdoc'
 Barclays.gdoc
'Colab Notebooks'
 Communication.gdoc
'Corporate Investment Report.gdoc'
'Corporate Investment Report 修改版本 - 柱形图 1 (1).gsheet'
'Corporate Investment Report 修改版本 - 柱形图 1.gsheet'
'Corporate Investment Report 修改版本 - 饼图 1 (1).gsheet'
'Corporate Investment Report 修改版本 - 饼图 1.gsheet'
 DATA
'DATA & Structure.gsheet'
 DM-Channels.gdoc
 Final.gdoc
'Final Report.gdoc'
'Investment_Banking_Yifei Li .pptx'
 Li_Yifei.pdf
'log of meeting.gdoc'
'Meeting 3 9.gdoc'
'Morgan Stanley MSFS Portfolio Analytics Role.gdoc'
 NY租房.gdoc
'Phase 2 Q1.gdoc'
'Phase 3 Report - ddl 2.16.gdoc'
'Phase 5 Report - ddl 3.30.gdoc'
'PM-Customer Journey Mapping .gdoc'
'Product Magt-Lecture1.gdoc'
 Project.gdoc
 Q3-Yifei.gdoc
'

In [78]:
!ls /content/drive/MyDrive/'DATA'

 BTCBUSD#F      BTCUSDT#F     'BTCUSDT#F 3'  'BTCUSDT#S 2'  'ETHUSDT#F 2'
'BTCBUSD#F 2'  'BTCUSDT#F 2'   BTCUSDT#S      ETHUSDT#F     'ETHUSDT#F 3'


In [93]:
path = "/content/drive/MyDrive/DATA/*/"
glob(os.path.join(path, '*ticker*.parquet'))

['/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_29_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_28_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_27_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_26_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_25_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_24_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_23_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_22_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_21_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCBUSD#F 2/2022_11_20_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCUSDT#F 2/2022_11_28_book_ticker_stream.parquet',
 '/content/drive/MyDrive/DATA/BTCUSDT#F 2/2022_11_27_book_ticker_stream.parquet',
 '/content/drive

In [94]:
import pandas as pd
from glob import glob
import os
import math
from tqdm import tqdm

In [95]:
def readFile(path):
    data = pd.read_parquet(path)
    data['ReceiveTime'] = data['ReceiveTime'].map(lambda x: math.floor(x))
    data.sort_index(ascending=True)
    data.drop_duplicates(subset=['ReceiveTime'], keep='first', inplace=True)
    return data

In [96]:
def process_ticker(path):
    data = readFile(path)
    data['price'] = (data['BidPx'] + data['AskPx'])/2
    price = data.loc[:, 'price']
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        match = data.loc[data['EventTime']>=row['ReceiveTime']+10000].copy()
        cur_price = row['price']
        match['price_change'] = (match['price']-cur_price)/cur_price
        print('price_change', match['price_change'])

In [97]:
def process_depth(path):
    data = readFile(path)
    data['price'] = (data['BidPx0']+data['AskPx0'])/2
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        match = data.loc[data['EventTime']>=row['ReceiveTime']+10000].copy()
        cur_price = row['price']
        match['price_change'] = (match['price']-cur_price)/cur_price

In [ ]:
if __name__ == '__main__':
    path = "/content/drive/MyDrive/DATA/*/"
    tickers = glob(os.path.join(path, '*ticker*.parquet'))
    depths = glob(os.path.join(path, '*depth*.parquet'))
    for ticker in tickers:
        process_ticker(ticker)
    for depth in depths:
        process_depth(depth)

  0%|          | 1/2856834 [00:00<554:06:46,  1.43it/s]

price_change 571       -0.000327
572       -0.000327
576       -0.000327
579       -0.000327
582       -0.000327
             ...   
3808174    0.013848
3808175    0.013848
3808176    0.013848
3808177    0.013848
3808178    0.013848
Name: price_change, Length: 2856429, dtype: float64


  0%|          | 2/2856834 [00:01<528:38:24,  1.50it/s]

price_change 1016      -0.000562
1017      -0.000562
1018      -0.000562
1019      -0.000562
1020      -0.000562
             ...   
3808174    0.013848
3808175    0.013848
3808176    0.013848
3808177    0.013848
3808178    0.013848
Name: price_change, Length: 2856115, dtype: float64


  0%|          | 3/2856834 [00:01<464:21:49,  1.71it/s]

...

  0%|          | 499/2856834 [03:26<342:21:42,  2.32it/s]

price_change 1224      -0.000056
1225      -0.000056
1226      -0.000056
1227      -0.000056
1228      -0.000056
             ...   
3808174    0.014412
3808175    0.014412
3808176    0.014412
3808177    0.014412
3808178    0.014412
Name: price_change, Length: 2855943, dtype: float64


  0%|          | 500/2856834 [03:27<350:15:41,  2.27it/s]

price_change 1224      -0.000056
1225      -0.000056
1226      -0.000056
1227      -0.000056
1228      -0.000056
             ...   
3808174    0.014412
3808175    0.014412
3808176    0.014412
3808177    0.014412
3808178    0.014412
Name: price_change, Length: 2855943, dtype: float64


  0%|          | 501/2856834 [03:27<353:59:20,  2.24it/s]